# **Overview**

This notebook benchmarks MONAI's implementation of Transformation and Warp/Resample layers against SimpleITK by reproducing some of the results from the [SimpleITK notebook](http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/Python_html/21_Transforms_and_Resampling.html).

# **Setup environment**

In [1]:
!pip install -q "monai-weekly[nibabel]"
import monai
from monai.apps.utils import download_url

!pip install -q "SimpleITK"
import SimpleITK as sitk

import torch
import os
import tempfile
import PIL
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt
from ipywidgets import interact, fixed

     |████████████████████████████████| 721 kB 4.0 MB/s 
     |████████████████████████████████| 48.4 MB 18 kB/s 


# **Download data**

For comparison against SimpleITK implementation [results](http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/Python_html/21_Transforms_and_Resampling.html), download the SimpleITK logo.

In [2]:
# create temporary directory
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(f"root dir is: {root_dir}")

# dict of file name and cooresponding urls
url = f"https://s3.amazonaws.com/simpleitk/public/notebooks/SHA512/f1b5ce1bf9d7ebc0bd66f1c3bc0f90d9e9798efc7d0c5ea7bebe0435f173355b27df632971d1771dc1fc3743c76753e6a28f6ed43c5531866bfa2b38f1b8fd46"
img_name = f"{root_dir}/SimpleITK.jpg"
download_url(url, img_name)

root dir is: /tmp/tmpwjxtrvnt


SimpleITK.jpg: 8.00kB [00:00, 10.3kB/s]

2022-01-04 23:55:11,211 - INFO - Downloaded: /tmp/tmpwjxtrvnt/SimpleITK.jpg
2022-01-04 23:55:11,213 - INFO - Expected md5 is None, skip md5 check for file /tmp/tmpwjxtrvnt/SimpleITK.jpg.


# **Transfromation**

This section displays the transformation functionalities in an interactive manner.

In [3]:
def transformation_display(img_path, tx, ty, theta):
    """
    transfrom a image by MOANI and SimpleITK and plot both transformed images
    """

    # monai transform
    monai_img = np.asarray(PIL.Image.open(img_name))
    monai_img = monai_img.transpose((2, 0, 1))
    transform = monai.transforms.Compose([
       monai.transforms.Affined(
           keys=["img"], 
           rotate_params=-theta, 
           translate_params=(ty,tx),
           padding_mode="zeros"
           ),
       monai.transforms.ScaleIntensityd(keys=["img"])
       ])
    monai_transformed = transform({"img": monai_img})["img"].transpose((1, 2, 0))


    # sitk transform
    sitk_img = sitk.ReadImage(img_path)
    euler2d_transform = sitk.Euler2DTransform()
    euler2d_transform.SetCenter(
        sitk_img.TransformContinuousIndexToPhysicalPoint(
            np.array(sitk_img.GetSize()) / 2.0
            )
        )
    euler2d_transform.SetTranslation((tx, ty))
    euler2d_transform.SetAngle(theta)
    sitk_transformed = sitk.Resample(sitk_img, euler2d_transform)

    # plot monai and sitk transformed results as subplots
    fig, ax = plt.subplots(2)
    ax[0].imshow(monai_transformed)
    ax[0].axis('off')
    ax[0].set_title("MONAI")
    ax[1].imshow(sitk.GetArrayFromImage(sitk_transformed))
    ax[1].axis('off')
    ax[1].set_title("SimpleITK")
    plt.show()
  

# generate the interactive display
interact(
    transformation_display, 
    img_path=fixed(f"{root_dir}/SimpleITK.jpg"), 
    tx=(-128.0, 128.0, 2.0), 
    ty=(-64.0, 64.0), 
    theta=(-np.pi/4.0,np.pi/4.0)
    );


interactive(children=(FloatSlider(value=0.0, description='tx', max=128.0, min=-128.0, step=2.0), FloatSlider(v…

# **Warping**

Warping distorts an `image` by `ddf` to give the `output`.

The following example randomly sample 10 physical points (locations) on an image and assign ascending displacements in y(H) direction accordingly.
Resampled intensities obtained by both MONAI and SimpleITK implementations are reported at all 10 points.

First, 10 physical points are randomly sampled from the downloaded SimpleITK logo.

In [4]:
# Read image
img_path= f"{root_dir}/SimpleITK.jpg"
monai_img = np.asarray(PIL.Image.open(img_name))
monai_img = monai_img.transpose((2, 0, 1))
monai_img = torch.tensor(monai_img, dtype=torch.float)
sitk_img = sitk.ReadImage(img_path)

width, height = sitk_img.GetSize()

# Generate random samples inside the image, we will obtain the intensity/color values at these points.
num_samples = 10
physical_points = np.array([np.random.randint(monai_img.shape[1:]) for _ in range(10)], dtype=np.float)  # (10, 2)


Currently, only spatial (4-D) and volumetric (5-D) input are supported by MONAI. In the following spatial (4-D) example, for an `image` of shape (batch, num_channels, H, W) and a `ddf` of shape (batch, 2, H, W), the output will have the same shape as `input`.

For each output location `output[b, :, h, w]`, the size-2 vector `ddf[b, :, h, w] = [h', w']` specifies coordinate displacement in y(H) and x(W) direaction from the input location `image[b, :, h, w]`. In other words, the value at `output[b, :, h, w]` should correspond to the value at `input[b, :, h + h', w + w']`

In [5]:
# initialise ddf as a zero matrix
ddf = torch.zeros(2, height, width).to(torch.float)  # (2, H, W)
# add displacement of y coordinate to sampled locations
for i, pnt in enumerate(physical_points.astype(np.long)):
  ddf[0, pnt[0], pnt[1]] += i
# initialise warp layer
warp = monai.networks.blocks.Warp(mode='bilinear', padding_mode='zeros')
# warp, note an batch dimension is added and removed during calculation
monai_resample = warp(
    monai_img.unsqueeze(0), ddf.unsqueeze(0).to(monai_img)
).squeeze(0)  # (3, H, W)

/usr/local/lib/python3.7/dist-packages/monai/networks/blocks/warp.py:65: UserWarning: monai.networks.blocks.Warp: Using PyTorch native grid_sample.
  warnings.warn("monai.networks.blocks.Warp: Using PyTorch native grid_sample.")


The same operation could be achived by the `Resample` method from SimpleITK. SimpleITK's procedural API provides three methods for performing resampling. Here we use 

`Resample(const Image &image1, const Image &referenceImage, Transform transform, InterpolatorEnum interpolator, double defaultPixelValue, PixelIDValueEnum outputPixelType)`

In [6]:
# Create an image of size [width, height]. The pixel type is irrelevant, as the image is 
# just defining the interpolation grid (sitkUInt8 has minimal memory footprint).
interp_grid_img = sitk.Image([width, height], sitk.sitkUInt8)

# initialise displacement
sitk_displacement_img = sitk.Image([width, height], sitk.sitkVectorFloat64, sitk_img.GetDimension())
# add displacement of y coordinate to sampled locations
for i, pnt in enumerate(physical_points):
  sitk_displacement_img[int(pnt[1]), int(pnt[0])] = np.array([0, i], dtype=np.float)

# select linear interpolater to match `bilinear` mode in MONAI
interpolator_enum = sitk.sitkLinear
# set default_output_pixel_value to 0.0 to match `zero` padding_mode in MONAI
default_output_pixel_value = 0.0
# set output_pixel_type
output_pixel_type = sitk.sitkVectorFloat32
# resample
sitk_resample = sitk.Resample(
    sitk_img, 
    interp_grid_img, 
    sitk.DisplacementFieldTransform(sitk_displacement_img), 
    interpolator_enum, 
    default_output_pixel_value, 
    output_pixel_type
    )

# turn resampled image into numpy array for later comparison
sitk_resample = sitk.GetArrayFromImage(sitk_resample)

Compare results from MONAI and SimpleITK. As could be observed, same intensities are predicted by both MONAI and SimpleITK implementation at all 10 points.

In [7]:
for i, pnt in enumerate(physical_points.astype(np.long)):
    print(f"at location {pnt}: original intensity {monai_img[:, pnt[0], pnt[1]]} " + 
      f"resampled to {monai_resample[:, pnt[0], pnt[1]]} by MONAI and {sitk_resample[pnt[0], pnt[1]]} by SITK")

at location [ 40 119]: original intensity tensor([254., 254., 255.])resampled to tensor([254.0000, 254.0000, 255.0000]) by MONAI and [254. 254. 255.] by SITK
at location [14 55]: original intensity tensor([255., 255., 255.])resampled to tensor([255., 255., 255.]) by MONAI and [255. 255. 255.] by SITK
at location [38 64]: original intensity tensor([255., 254., 249.])resampled to tensor([252., 239., 231.]) by MONAI and [252. 239. 231.] by SITK
at location [ 39 124]: original intensity tensor([ 45.,  90., 155.])resampled to tensor([53.9998, 63.0002, 96.0006]) by MONAI and [54. 63. 96.] by SITK
at location [14 66]: original intensity tensor([255., 255., 255.])resampled to tensor([255., 255., 255.]) by MONAI and [255. 255. 255.] by SITK
at location [15 97]: original intensity tensor([255., 255., 255.])resampled to tensor([255., 255., 255.]) by MONAI and [255. 255. 255.] by SITK
at location [ 19 155]: original intensity tensor([253., 253., 255.])resampled to tensor([252., 254., 253.]) by MON